In [1]:
use Dan :ALL;

In [2]:
# Read .csv file (Text::CSV)

use Text::CSV;
my @lines  = csv( in => 'DemographicData.csv' );

my \stats = DataFrame.new( columns => @lines[0], data => @lines[1..*-1] );
~stats[0..5]^;      # head

    Country Name          Country Code  Birth rate  Internet users  Income Group        
 0  Aruba                 ABW           10.244      78.9            High income         
 1  Afghanistan           AFG           35.253      5.9             Low income          
 2  Angola                AGO           45.985      19.1            Upper middle income 
 3  Albania               ALB           12.877      57.2            Upper middle income 
 4  United Arab Emirates  ARE           11.044      88              High income         
 5  Argentina             ARG           17.716      59.9            High income         

These two read csv options illustrate how the close relationship between raku code, modules and Dan objects makes it easy for programmers to unpack what is going on and to step in. 

What if you encounter non-standard character "noise" in input files, or non-standard quoting constructs? ... just write your own csv Regex - or even a whole Grammar! Also note the use of the << $_ >> which turns a Str into an [Allomorph](https://docs.raku.org/syntax/%3C%3C%20%3E%3E) such as IntStr, RatStr, NumStr...

What if you want RFC4180 compliance or unicode separators or unusual eol markers? ... then the comprehensive raku [Text::CSV](https://github.com/Tux/CSV) module is for you.

In [3]:
# Read .csv file (Regex)

#| comb match anything except " and , -or- take a quoted string
my Regex \csv = rx/ <-[",]>+ | <["]> ~ <["]>  <-["]>+ /;

my @lines   = 'DemographicData.csv'.IO.lines;

my @columns = @lines.shift.comb( csv );
my @data    = @lines.map(*.comb( csv ).map({<< $_ >>}).Array);

my \stats2   = DataFrame.new( :@columns, :@data );
~stats2[0..5]^;      # head

    Country Name          Country Code  Birth rate  Internet users  Income Group        
 0  Aruba                 ABW           10.244      78.9            High income         
 1  Afghanistan           AFG           35.253      5.9             Low income          
 2  Angola                AGO           45.985      19.1            Upper middle income 
 3  Albania               ALB           12.877      57.2            Upper middle income 
 4  United Arab Emirates  ARE           11.044      88              High income         
 5  Argentina             ARG           17.716      59.9            High income         

In [4]:
# '~' makes a Str, '^' makes a DataFrame from rows

~stats[*-5..*-1]^;    # tail

      Country Name      Country Code  Birth rate  Internet users  Income Group        
 190  Yemen, Rep.       YEM           32.947      20              Lower middle income 
 191  South Africa      ZAF           20.85       46.5            Upper middle income 
 192  Congo, Dem. Rep.  COD           42.394      2.2             Low income          
 193  Zambia            ZMB           40.471      15.4            Lower middle income 
 194  Zimbabwe          ZWE           35.715      18.5            Low income          

In [5]:
stats.shape;

(195 5)

In [6]:
stats.elems;

195

In [7]:
stats.columns.elems;  

5

In [8]:
stats.columns = <CountryName CountryCode BirthRate InternetUsers IncomeGroup> Z=> 0..∞;
say stats.columns;     #Hash (unordered)
say stats.cx;          #Seq    (ordered)

{BirthRate => 2, CountryCode => 1, CountryName => 0, IncomeGroup => 4, InternetUsers => 3}
(CountryName CountryCode BirthRate InternetUsers IncomeGroup)


In [9]:
# raku Dan stores the original data types & converts on the fly (hmmm)
stats.dtypes;

CountryName => Str
CountryCode => Str
BirthRate => Str
InternetUsers => Str
IncomeGroup => Str

In [11]:
stats.describe;      #FIXME

Cannot convert string to number: base-10 number must begin with valid digits or '.' in '⏏Aruba' (indicated by ⏏)

In [12]:
~stats[0..5]<CountryName BirthRate>;

    CountryName           BirthRate 
 0  Aruba                 10.244    
 1  Afghanistan           35.253    
 2  Angola                45.985    
 3  Albania               12.877    
 4  United Arab Emirates  11.044    
 5  Argentina             17.716    

In [13]:
~stats[185..*-1]^;       # Range operator (start .. end)

      CountryName            CountryCode  BirthRate  InternetUsers  IncomeGroup         
 185  Virgin Islands (U.S.)  VIR          10.7       45.3           High income         
 186  Vietnam                VNM          15.537     43.9           Lower middle income 
 187  Vanuatu                VUT          26.739     11.3           Lower middle income 
 188  West Bank and Gaza     PSE          30.394     46.6           Lower middle income 
 189  Samoa                  WSM          26.172     15.3           Lower middle income 
 190  Yemen, Rep.            YEM          32.947     20             Lower middle income 
 191  South Africa           ZAF          20.85      46.5           Upper middle income 
 192  Congo, Dem. Rep.       COD          42.394     2.2            Low income          
 193  Zambia                 ZMB          40.471     15.4           Lower middle income 
 194  Zimbabwe               ZWE          35.715     18.5           Low income          

In [14]:
#Reverse order
~stats[10,*-1...5]^;     # Sequence operator (start,operation ... end)

     CountryName          CountryCode  BirthRate  InternetUsers  IncomeGroup         
 10  Azerbaijan           AZE          18.3       58.7           Upper middle income 
 9   Austria              AUT          9.4        80.6188        High income         
 8   Australia            AUS          13.2       83             High income         
 7   Antigua and Barbuda  ATG          16.447     63.4           High income         
 6   Armenia              ARM          13.308     41.9           Lower middle income 
 5   Argentina            ARG          17.716     59.9           High income         

In [15]:
#Every 20th item
~stats[0,20...stats.elems]^;

      CountryName  CountryCode  BirthRate  InternetUsers  IncomeGroup         
 0    Aruba        ABW          10.244     78.9           High income         
 20   Belarus      BLR          12.5       54.17          Upper middle income 
 40   Costa Rica   CRI          15.022     45.96          Upper middle income 
 60   Gabon        GAB          30.555     9.2            Upper middle income 
 80   India        IND          20.291     15.1           Lower middle income 
 100  Libya        LBY          21.425     16.5           Upper middle income 
 120  Mozambique   MOZ          39.705     5.4            Low income          
 140  Poland       POL          9.6        62.8492        High income         
 160  Suriname     SUR          18.455     37.4           Upper middle income 
 180  Uruguay      URY          14.374     57.69          High income         

In [16]:
my $calcs = Series.new( stats[*]<BirthRate> >>*<< stats[*]<InternetUsers> );

stats.splice( :ax, *, *, [MyCalc => $calcs] );

~stats[0..5]^;

    CountryName           CountryCode  BirthRate  InternetUsers  IncomeGroup          MyCalc    
 0  Aruba                 ABW          10.244     78.9           High income          808.2516  
 1  Afghanistan           AFG          35.253     5.9            Low income           207.9927  
 2  Angola                AGO          45.985     19.1           Upper middle income  878.3135  
 3  Albania               ALB          12.877     57.2           Upper middle income  736.5644  
 4  United Arab Emirates  ARE          11.044     88             High income          971.872   
 5  Argentina             ARG          17.716     59.9           High income          1061.1884 

In [17]:
#FIXME make .grep no :"in place"

()